First things first... We always need to make sure the requirements are installed into the notebooks so we need to find the working directory too.

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\welsh\\Desktop\\PF_SITE\\PredTesting\\PredTesting'

As I haven't put the notebooks into their own folder the working directory should be correct. If the working directory doesn't match where my jupreqs.txt file is then edit the path in the code below to point to where the file is. Otherwise just run it and wait a couple of minutes.

In [2]:
%pip install -r jupreqs.txt

Note: you may need to restart the kernel to use updated packages.


Now we need to fetch/import our data. For this we are going to look at vegetable prices over time. I have downloaded [this](https://www.kaggle.com/datasets/ksamiksha19/vegetable-prices) and imported it into the directory by hand (PredTesting\inputs\datasets\raw\prices.csv).

Once that's done I have a first look into the data to make sure it's ready for my use.

In [6]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/raw/prices.csv")
df.head(20)

,Price Dates,Bhindi (Ladies finger),Tomato,Onion,Potato,Brinjal,Garlic,Peas,Methi,Green Chilli,Elephant Yam (Suran)
0,01-01-2023,35.0,18,22.0,20,30,50,25,8,45.0,25
1,02-01-2023,35.0,16,22.0,20,30,55,25,7,40.0,25
2,03-01-2023,35.0,16,21.0,20,30,55,25,7,40.0,25
3,04-01-2023,30.0,16,21.0,22,25,55,25,7,40.0,25
4,08-01-2023,35.0,16,20.0,21,25,55,22,6,35.0,25
5,11-01-2023,35.0,16,18.0,24,25,55,23,6,35.0,30
6,12-01-2023,40.0,16,18.0,22,30,65,23,6,35.0,30
7,15-01-2023,42.0,16,17.0,22,25,65,23,7,35.0,30
8,17-01-2023,35.0,16,18.0,22,30,65,22,10,40.0,35
9,22-01-2023,45.0,16,18.0,22,40,65,25,9,40.0,35


Lets check the data types we are working with.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Price Dates             287 non-null    object 
 1   Bhindi (Ladies finger)  287 non-null    float64
 2   Tomato                  287 non-null    int64  
 3   Onion                   287 non-null    float64
 4   Potato                  287 non-null    int64  
 5   Brinjal                 287 non-null    int64  
 6   Garlic                  287 non-null    int64  
 7   Peas                    287 non-null    int64  
 8   Methi                   287 non-null    int64  
 9   Green Chilli            287 non-null    float64
 10  Elephant Yam (Suran)    287 non-null    int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 24.8+ KB


So this looks like it has no missing entries which is great! But let's double check this just to make sure.

In [9]:
vars_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_missing_data

[]

In [10]:
from ydata_profiling import ProfileReport
if vars_missing_data:
    profile = ProfileReport(df=df[vars_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

C:\Users\welsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\welsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for detail

There are no variables with missing data


Despite changing no data, it is good practice to save this initial collected + processed data into a new file.

In [12]:
import os
try:
  os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

df.to_csv(f"outputs/datasets/collection/VegPrices.csv",index=False)

[WinError 183] Cannot create a file when that file already exists: 'outputs/datasets/collection'
